In [19]:
import psycopg2

def execute_query(query, params=None):
    try:
        # Replace the following values with your actual database credentials
        conn = psycopg2.connect(
            database="db_campusv2",
            user="postgres",
            password="MerSer01",
            host="localhost",
            port="5432"  # Default PostgreSQL port
        )

        with conn.cursor() as cur:
            cur.execute(query, params)
            # Fetch and process data if needed

        # Commit the changes (if any)
        conn.commit()

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close the connection in the 'finally' block to ensure it is always closed
        if conn:
            conn.close()

# Example: Inserting data into a table
insert_query = "INSERT INTO your_table (column1, column2) VALUES (%s, %s)"
insert_params = ('value1', 'value2')
execute_query(insert_query, insert_params)

# Example: Selecting data from a table
select_query = "SELECT * FROM your_table"
execute_query(select_query)


Error: relation "your_table" does not exist
LINE 1: INSERT INTO your_table (column1, column2) VALUES ('value1', ...
                    ^

Error: relation "your_table" does not exist
LINE 1: SELECT * FROM your_table
                      ^



In [1]:
import psycopg2


In [37]:
from passlib.hash import bcrypt
import psycopg2

# Database connection details
database_name = "db_campusv2"
user = "postgres"
password = "MerSer01"
host = "localhost"
port = "5432"

# Construct the database URL
db_url = f"postgresql://{user}:{password}@{host}:{port}/{database_name}"

# Database transactions class
class db_transactions:
    def __init__(self, db_url):
        self.conn = psycopg2.connect(db_url)
        self.cur = self.conn.cursor()

    def run_query(self, query, data=None):
        try:
            self.cur.execute(query, data)
            self.conn.commit()
        except Exception as e:
            print(f"Error: {e}")
            self.conn.rollback()

    def fetch_data(self, query, data=None):
        try:
            self.cur.execute(query, data)
            return self.cur.fetchall()
        except Exception as e:
            print(f"Error: {e}")
            return None

    def close_connection(self):
        self.cur.close()
        self.conn.close()


# Login function
def login(db_trans, email, entered_password):
    # Retrieve hashed password from the 'usertable' based on email
    stored_password_data = db_trans.fetch_data("SELECT password FROM usertable WHERE email = %s", (email,))


# Signup function
def signup(db_trans, signup_data):
    # Unpack tuple into variables
    email, plain_password, first_name, last_name, phone, city, gender, birthdate, user_type, application_id = signup_data


    existing_user = db_trans.fetch_data("SELECT user_id FROM usertable WHERE email = %s", (email,))

    if existing_user:
        print("Email is already taken. Please choose another email.")
        return
    
    # Hash the plain password
    hashed_password = bcrypt.hash(plain_password)

    # Insert user data into 'usertable'
    db_trans.run_query("""
        INSERT INTO usertable (
            email, password, first_name, last_name, phone, city, gender, birthdate, user_type, status, application_id
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (email, hashed_password, first_name, last_name, phone, city, gender, birthdate, user_type, True, application_id))

    # Retrieve user_id based on email
    user_id = db_trans.fetch_data("SELECT user_id FROM usertable WHERE email = %s", (email,))[0][0]

    # Insert hashed password into 'hashedpasswords' table
    db_trans.run_query("""
        INSERT INTO hashedpasswords (user_id, hashed_password) VALUES (%s, %s)
    """, (user_id, hashed_password))

# Assign values to variables
email = "serkanbakisgan1@gmail.com"
plain_password = "AbCn12**"
first_name = "Serkan"
last_name = "Bakisgan"
phone = "015202819030"
city = "Gaziantep"
gender = "Male"
birthdate = "1980-04-30"
user_type = "Student"
application_id = None

# Create a tuple with the variables
signup_data = (email, plain_password, first_name, last_name, phone, city, gender, birthdate, user_type, application_id)

# Create an instance of db_transactions
db_trans = db_transactions(db_url)

# Call the signup function with the tuple variable
signup(db_trans, signup_data)

# Close the database connection
db_trans.close_connection()


Email is already taken. Please choose another email.


In [39]:
email="serkanbakisgan1@gmail.com"
password="AbCn12**"

db_trans = db_transactions(db_url)

def login(db_trans, email, entered_password):
    # Retrieve hashed password from the 'usertable' based on email
    stored_password_data = db_trans.fetch_data("SELECT password FROM usertable WHERE email = %s", (email,))

    if stored_password_data:
        stored_hashed_password = stored_password_data[0][0]

        # Verify if the entered password matches the stored hashed password
        password_match = bcrypt.verify(entered_password, stored_hashed_password)

        if password_match:
            print("Login successful.")
        else:
            print("Incorrect password.")
    else:
        print("User not found.")


login(db_trans,email,password)

Login successful.


In [40]:
global_variable = "I am a global variable"

def example_function():
    # Local variable
    local_variable = "I am a local variable"
    
    # Reassign values
    global global_variable  # Reassign global variable
    global_variable = "Modified global variable"
    
    local_variable = "Modified local variable"  # Reassign local variable
    
    # Print the modified values
    print("Inside the function - Modified Global variable:", global_variable)
    print("Inside the function - Modified Local variable:", local_variable)

# Print the initial values
print("Outside the function - Initial Global variable:", global_variable)

# Try to access the local variable outside the function (will result in an error)
try:
    print("Outside the function - Initial Local variable:", local_variable)
except NameError as e:
    print("Outside the function - Error:", e)

# Call the function
example_function()

# Print the modified global variable outside the function
print("Outside the function - Modified Global variable:", global_variable)

Outside the function - Initial Global variable: I am a global variable
Outside the function - Error: name 'local_variable' is not defined
Inside the function - Modified Global variable: Modified global variable
Inside the function - Modified Local variable: Modified local variable
Outside the function - Modified Global variable: Modified global variable


In [41]:
import psycopg2

database_name = "db_campusv2"
user = "postgres"
password = "MerSer01"
host = "localhost"
port = "5432"
db_url = f"postgresql://{user}:{password}@{host}:{port}/{database_name}"

conn = psycopg2.connect(db_url)
cur = conn.cursor()

cur.execute("""UPDATE usertable
            SET last_name = %s
            WHERE email = %s
            """, (
                        "Bakiskan",
                        "serkanbakisgan1@gmail.com"
                    ))
conn.commit()
cur.close()
conn.close()


In [ ]:
        try:
            conn = psycopg2.connect(db_url)
            cur = conn.cursor()
            cur.execute("""
                    SELECT password FROM usertable
                    WHERE email = %s
                """, (email,))
            stored_password_data = cur.value
